# Create train and test from dataset

Use 90% of images as train data and 10% of images as test data

In [2]:
import os
import shutil
import random

In [3]:
# Path to your dataset directory
dataset_dir = 'E:\\dataset\\cattle\\Indian_bovine_breeds'  # Replace with your actual path

target_dir = 'E:\\AI\\freerolls\\dataset'

# Create train/test directories
train_dir = os.path.join(target_dir, 'train')
test_dir = os.path.join(target_dir, 'test')
os.makedirs(train_dir, exist_ok=True)  # Create if not exist
os.makedirs(test_dir, exist_ok=True)

In [12]:
# Get category directories
category_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

# Process each category
for category_dir in category_dirs:
    category_path = os.path.join(dataset_dir, category_dir)
    image_files = [f for f in os.listdir(category_path)]  # Adjust for your image formats

    # Shuffle images randomly
    random.shuffle(image_files)

    # Calculate number of images for testing (10%)
    num_test_images = int(0.10 * len(image_files))

    # Move test images
    for image_file in image_files[:num_test_images]:
        src_path = os.path.join(category_path, image_file)
        dst_path = os.path.join(test_dir, category_dir, image_file)
        os.makedirs(os.path.join(test_dir, category_dir), exist_ok=True) # Create category folder in test dir if not exist
        shutil.copy(src_path, dst_path)

    # Move remaining train images
    for image_file in image_files[num_test_images:]:
        src_path = os.path.join(category_path, image_file)
        dst_path = os.path.join(train_dir, category_dir, image_file)
        os.makedirs(os.path.join(train_dir, category_dir), exist_ok=True)
        shutil.copy(src_path, dst_path)

    # The rest remain in the train folder (no need to move explicitly)
    print(f"Processed category: {category_dir}")

Processed category: Alambadi
Processed category: Amritmahal
Processed category: Ayrshire
Processed category: Banni
Processed category: Bargur
Processed category: Bhadawari
Processed category: Brown_Swiss
Processed category: Dangi
Processed category: Deoni
Processed category: Gir
Processed category: Guernsey
Processed category: Hallikar
Processed category: Hariana
Processed category: Holstein_Friesian
Processed category: Jaffrabadi
Processed category: Jersey
Processed category: Kangayam
Processed category: Kankrej
Processed category: Kasargod
Processed category: Kenkatha
Processed category: Kherigarh
Processed category: Khillari
Processed category: Krishna_Valley
Processed category: Malnad_gidda
Processed category: Mehsana
Processed category: Murrah
Processed category: Nagori
Processed category: Nagpuri
Processed category: Nili_Ravi
Processed category: Nimari
Processed category: Ongole
Processed category: Pulikulam
Processed category: Rathi
Processed category: Red_Dane
Processed categor

# Create function for class mapping

In [4]:
def get_class_mapping(path):
    """
    Creates a class mapping dictionary from folder names in ascending order.

    Args:
        path (str): The path to the directory containing class folders.

    Returns:
        dict: A dictionary mapping class folder names to integer indices.
    """

    class_folders = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    class_folders.sort()  # Sort folder names in ascending order

    class_mapping = {}
    for index, folder_name in enumerate(class_folders):
        class_mapping[folder_name] = index

    return class_mapping

Get class_mapping

In [5]:
class_mapping = get_class_mapping(dataset_dir)
class_mapping

{'Alambadi': 0,
 'Amritmahal': 1,
 'Ayrshire': 2,
 'Banni': 3,
 'Bargur': 4,
 'Bhadawari': 5,
 'Brown_Swiss': 6,
 'Dangi': 7,
 'Deoni': 8,
 'Gir': 9,
 'Guernsey': 10,
 'Hallikar': 11,
 'Hariana': 12,
 'Holstein_Friesian': 13,
 'Jaffrabadi': 14,
 'Jersey': 15,
 'Kangayam': 16,
 'Kankrej': 17,
 'Kasargod': 18,
 'Kenkatha': 19,
 'Kherigarh': 20,
 'Khillari': 21,
 'Krishna_Valley': 22,
 'Malnad_gidda': 23,
 'Mehsana': 24,
 'Murrah': 25,
 'Nagori': 26,
 'Nagpuri': 27,
 'Nili_Ravi': 28,
 'Nimari': 29,
 'Ongole': 30,
 'Pulikulam': 31,
 'Rathi': 32,
 'Red_Dane': 33,
 'Red_Sindhi': 34,
 'Sahiwal': 35,
 'Surti': 36,
 'Tharparkar': 37,
 'Toda': 38,
 'Umblachery': 39,
 'Vechur': 40}

# Training data

In [6]:
import numpy as np
import cv2
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten , Conv2D, MaxPool2D
import matplotlib.pyplot as plt

In [7]:
width = 224
num_classes = 41
train_path = 'E:\\AI\\freerolls\\dataset\\train/'
test_path = 'E:\\AI\\freerolls\\dataset\\test/'
train_img = [train_path+f for f in listdir(train_path)]
test_img = [test_path+f for f in listdir(test_path)]

In [8]:
train_img,test_img

(['E:\\AI\\freerolls\\dataset\\train/Alambadi',
  'E:\\AI\\freerolls\\dataset\\train/Amritmahal',
  'E:\\AI\\freerolls\\dataset\\train/Ayrshire',
  'E:\\AI\\freerolls\\dataset\\train/Banni',
  'E:\\AI\\freerolls\\dataset\\train/Bargur',
  'E:\\AI\\freerolls\\dataset\\train/Bhadawari',
  'E:\\AI\\freerolls\\dataset\\train/Brown_Swiss',
  'E:\\AI\\freerolls\\dataset\\train/Dangi',
  'E:\\AI\\freerolls\\dataset\\train/Deoni',
  'E:\\AI\\freerolls\\dataset\\train/Gir',
  'E:\\AI\\freerolls\\dataset\\train/Guernsey',
  'E:\\AI\\freerolls\\dataset\\train/Hallikar',
  'E:\\AI\\freerolls\\dataset\\train/Hariana',
  'E:\\AI\\freerolls\\dataset\\train/Holstein_Friesian',
  'E:\\AI\\freerolls\\dataset\\train/Jaffrabadi',
  'E:\\AI\\freerolls\\dataset\\train/Jersey',
  'E:\\AI\\freerolls\\dataset\\train/Kangayam',
  'E:\\AI\\freerolls\\dataset\\train/Kankrej',
  'E:\\AI\\freerolls\\dataset\\train/Kasargod',
  'E:\\AI\\freerolls\\dataset\\train/Kenkatha',
  'E:\\AI\\freerolls\\dataset\\train/Kherig

In [21]:
def img2data(path):
    rawImgs = []
    labels = []

    for imagePath in path:
        for item in tqdm(listdir(imagePath)):
            file = join(imagePath, item)
            if file[-1] == "g":
                img = cv2.imread(file, cv2.COLOR_BGR2RGB)

                if len(img.shape) != 3:
                    continue

                img = cv2.resize(img, (width, width), interpolation=cv2.INTER_CUBIC)
                rawImgs.append(img)

                l = imagePath.split('/').pop()
                label_index = class_mapping[l]  # Get class index from dictionary
                one_hot_label = np.zeros(41)  # One-hot vector of length 41
                one_hot_label[label_index] = 1
                labels.append(one_hot_label)

    return rawImgs, labels

In [22]:
x_train, y_train = img2data(train_img)
x_test, y_test = img2data(test_img)

100%|██████████| 27/27 [00:00<00:00, 53.15it/s]


In [23]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Setup and training mode

Create new model using tranform learning from MobileNetV3Large

In [24]:
base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet')
base_model.trainable = False
num_classes = len(class_mapping)

model = Sequential([
    base_model,
    Conv2D(224, 3, activation='relu'),
    Dense(16),
    Flatten(),
    Dense(num_classes, activation='softmax')
])

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics= ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', baseline=0.4, verbose=1, patience=50)

In [26]:
batch_size = 32
epochs = 100
history = model.fit(x_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(x_test, y_test), callbacks=[es])

Epoch 1/100


181/181 [==============================] - 82s 425ms/step - loss: 3.5670 - accuracy: 0.0984 - val_loss: 3.4835 - val_accuracy: 0.1087
Epoch 2/100
181/181 [==============================] - 86s 475ms/step - loss: 3.4433 - accuracy: 0.1128 - val_loss: 3.4005 - val_accuracy: 0.1312
Epoch 3/100
181/181 [==============================] - 98s 540ms/step - loss: 3.3731 - accuracy: 0.1360 - val_loss: 3.3024 - val_accuracy: 0.1528
Epoch 4/100
181/181 [==============================] - 131s 724ms/step - loss: 3.3019 - accuracy: 0.1482 - val_loss: 3.2507 - val_accuracy: 0.1565
Epoch 5/100
181/181 [==============================] - 128s 708ms/step - loss: 3.2425 - accuracy: 0.1643 - val_loss: 3.2093 - val_accuracy: 0.1771
Epoch 6/100
181/181 [==============================] - 120s 661ms/step - loss: 3.1883 - accuracy: 0.1761 - val_loss: 3.1397 - val_accuracy: 0.1856
Epoch 7/100
181/181 [==============================] - 116s 644ms/step - loss: 3.1291 - accuracy: 0.1870 - val_loss: 3.

In [27]:
model.save("./indian_bovine_breeds_v1.keras")

In [33]:
# Get the class names in the desired order
class_names = list(class_mapping.keys())
class_names[15]

'Jersey'

In [31]:
testImg = [test_path+f for f in listdir(test_path)]
rimg = []
for imagePath in (testImg):
    for i,item in enumerate(listdir(imagePath)):
        file = join(imagePath, item)
        if file[-1] =='g':
            try:
                img = cv2.imread(file , cv2.COLOR_BGR2RGB)
                ori = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img ,(width,width))
                rimg = np.array(img)
                rimg = rimg.astype('float32')
                rimg /= 255
                rimg = np.reshape(rimg ,(1,224,224,3))
                predict = model.predict(rimg)

                # Print predict result
                probability = np.max(predict)

                # Hot encode to label
                hot_encode = np.argmax(predict)

                result = class_names[hot_encode]
                # Print path
                print('===================')
                print('probability:'+str(probability))
                print('real:'+str(item))
                print('predict:'+str(result))
                print('-------------------')
            except cv2.error as e:
                print(f"Error processing image {file}: {e}")

            except IndexError as e:
                print(f"Index error for image {file}: {e}")

            except Exception as e:  # Catch any other unexpected errors
                print(f"Unexpected error for image {file}: {e}")

            # # Print predict result on image
            # plt.title(str(result) + ': ' + str(probability))


            # plt.imshow(ori)
            # plt.show()

1/1 [==============================] - 2s 2s/step
probability:0.8875653
real:Alambadi_10.png
predict:Alambadi
-------------------
1/1 [==============================] - 0s 39ms/step
probability:0.6444049
real:Alambadi_13.png
predict:Alambadi
-------------------
1/1 [==============================] - 0s 40ms/step
probability:0.31251007
real:Alambadi_16.jpg
predict:Alambadi
-------------------
1/1 [==============================] - 0s 53ms/step
probability:0.22940381
real:Alambadi_3.png
predict:Alambadi
-------------------
1/1 [==============================] - 0s 42ms/step
probability:0.88106596
real:Alambadi_31.png
predict:Alambadi
-------------------
1/1 [==============================] - 0s 159ms/step
probability:0.9837234
real:Alambadi_36.png
predict:Alambadi
-------------------
1/1 [==============================] - 0s 39ms/step
probability:0.7067991
real:Alambadi_4.png
predict:Alambadi
-------------------
1/1 [==============================] - 0s 40ms/step
probability:0.1230019
re